**<h1 align="center">Image Exploration v1.3</h1>**

## **[IMAGE] 3-Column Pipeline Viewer & Exporter**
- **Column 1**: Overlay images (segmentation visualization)
- **Column 2**: Binary mask images (lung boundaries)  
- **Column 3**: Final processed images (crop/resize results)
- **Synchronized Display**: All columns show the same FileID for comparison
- **Flexible Filtering**: Display specific FileIDs or limit number of rows
- **NEW**: Export functionality with batch processing and progress tracking

In [ ]:
# Global Configuration Variables

# CSV Files
CSV_FOLDER = "/home/pyuser/wkdir/CSI-Predictor/data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"
IMPORT_COLUMNS = []
CHUNK_SIZE = 50000

# Download parameters  
DOWNLOAD_PATH = '/home/pyuser/wkdir/CSI-Predictor/data/Paradise_Test_DICOMs'
EXPORT_METADATA = True
ARCHIMED_METADATA_FILE = 'DICOM_Metadata.csv'
CONVERT = True

# Conversion parameters
BATCH_SIZE = 50
BIT_DEPTH = 8
CREATE_SUBFOLDERS = False
DELETE_DICOM = True
MONOCHROME = 1

# Enhanced Parameters
TARGET_SIZE = (518, 518)
PRESERVE_ASPECT_RATIO = True

# IMAGE EXPLORATION SETTINGS V1.3
MASKS_PATH = '/home/pyuser/data/Paradise_Masks'
IMAGES_PATH = '/home/pyuser/data/Paradise_Test_Images'
EXPORT_DIR = '/home/pyuser/data/Paradise_Exports'
FILEID_FILTER = []  # FileIDs to ignore during processing

print("Image Exploration v1.3 configuration loaded!")
print(f"Masks folder: {MASKS_PATH}")
print(f"Images folder: {IMAGES_PATH}")
print(f"Export folder: {EXPORT_DIR}")
print(f"Target size: {TARGET_SIZE}")

In [ ]:
# Core dependencies
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from IPython.display import display, HTML
from tqdm.auto import tqdm

# Colors for output
ANSI = {
    'R': '\033[91m', 'G': '\033[92m', 'B': '\033[94m', 'Y': '\033[93m',
    'W': '\033[0m', 'M': '\033[95m', 'C': '\033[96m'
}

print(f"{ANSI['G']}Core dependencies loaded{ANSI['W']}")

In [ ]:
def display_pipeline_images(nb_images_to_display=None, filter=None):
    """
    Display images in 3-column pipeline format: Overlay → Mask → Final Result
    
    Parameters:
    -----------
    nb_images_to_display : int, optional
        Number of image rows to display (each row contains 3 images)
    filter : list, optional
        List of FileIDs to display. If None, display all available FileIDs
    """
    
    print(f"{ANSI['C']}Loading 3-column pipeline view...{ANSI['W']}")
    
    # Step 1: Get all available FileIDs from final images (IMAGES_PATH)
    final_images = glob.glob(os.path.join(IMAGES_PATH, "*.png"))
    
    if not final_images:
        print(f"{ANSI['R']}No final images found in: {IMAGES_PATH}{ANSI['W']}")
        return
    
    # Extract FileIDs from final image filenames (FileID.png)
    available_file_ids = []
    for img_path in final_images:
        filename = os.path.basename(img_path)
        file_id = os.path.splitext(filename)[0]  # Remove .png extension
        available_file_ids.append(file_id)
    
    available_file_ids.sort()
    print(f"{ANSI['B']}Found {len(available_file_ids)} FileIDs in final images{ANSI['W']}")
    
    # Step 2: Apply filter if provided
    if filter:
        filtered_file_ids = [fid for fid in available_file_ids if fid in filter]
        print(f"{ANSI['Y']}Filter applied: {len(filtered_file_ids)}/{len(available_file_ids)} FileIDs match filter{ANSI['W']}")
        file_ids_to_display = filtered_file_ids
    else:
        file_ids_to_display = available_file_ids
        print(f"{ANSI['G']}No filter applied - using all available FileIDs{ANSI['W']}")
    
    if not file_ids_to_display:
        print(f"{ANSI['Y']}No FileIDs to display after filtering{ANSI['W']}")
        return
    
    # Step 3: Limit number of images if specified
    if nb_images_to_display and nb_images_to_display < len(file_ids_to_display):
        file_ids_to_display = file_ids_to_display[:nb_images_to_display]
        print(f"{ANSI['C']}Limited to {nb_images_to_display} image rows{ANSI['W']}")
    
    num_rows = len(file_ids_to_display)
    print(f"{ANSI['G']}Display grid: {num_rows} rows × 3 columns{ANSI['W']}")
    
    # Step 4: Create matplotlib figure with 3 columns
    fig, axes = plt.subplots(num_rows, 3, figsize=(15, 5*num_rows))
    
    # Handle single row case
    if num_rows == 1:
        axes = axes.reshape(1, -1)
    
    # Column headers
    column_titles = ['Overlay', 'Mask', 'Final']
    
    # Step 5: Display images for each FileID
    for row_idx, file_id in enumerate(file_ids_to_display):
        print(f"{ANSI['C']}Processing FileID: {file_id}{ANSI['W']}")
        
        # Find corresponding images for this FileID
        overlay_path = None
        mask_path = None
        final_path = os.path.join(IMAGES_PATH, f"{file_id}.png")
        
        # Search for overlay and mask files containing this FileID
        mask_files = glob.glob(os.path.join(MASKS_PATH, "*.png"))
        for mask_file in mask_files:
            filename = os.path.basename(mask_file)
            if file_id in filename:
                if "overlay" in filename:
                    overlay_path = mask_file
                elif "mask" in filename and "overlay" not in filename:
                    mask_path = mask_file
        
        # Display images in 3 columns
        image_paths = [overlay_path, mask_path, final_path]
        image_labels = [f"{file_id}_overlay", f"{file_id}_mask", file_id]
        
        for col_idx, (img_path, img_label) in enumerate(zip(image_paths, image_labels)):
            try:
                if img_path and os.path.exists(img_path):
                    # Load and display image
                    img = Image.open(img_path)
                    axes[row_idx, col_idx].imshow(img, cmap='gray' if img.mode == 'L' else None)
                    axes[row_idx, col_idx].set_title(f"{column_titles[col_idx]}\n{img_label}", fontsize=10, pad=10)
                else:
                    # Missing image
                    axes[row_idx, col_idx].text(0.5, 0.5, f'Missing\n{column_titles[col_idx]}\n{img_label}', 
                                               ha='center', va='center', transform=axes[row_idx, col_idx].transAxes,
                                               fontsize=12, color='red')
                    axes[row_idx, col_idx].set_title(f"{column_titles[col_idx]}\n{img_label} (MISSING)", fontsize=10, pad=10)
                
                axes[row_idx, col_idx].axis('off')
                
            except Exception as e:
                # Handle broken images
                axes[row_idx, col_idx].text(0.5, 0.5, f'Error loading\n{img_label}', 
                                           ha='center', va='center', transform=axes[row_idx, col_idx].transAxes,
                                           fontsize=12, color='red')
                axes[row_idx, col_idx].set_title(f"{column_titles[col_idx]}\n{img_label} (ERROR)", fontsize=10, pad=10)
                axes[row_idx, col_idx].axis('off')
                print(f"{ANSI['R']}Error loading {img_path}: {e}{ANSI['W']}")
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
    
    # Summary
    print(f"\n{ANSI['G']}Successfully displayed {num_rows} image rows (pipeline view){ANSI['W']}")
    if filter:
        print(f"{ANSI['C']}Filter: {len(filter)} FileIDs specified{ANSI['W']}")
    if nb_images_to_display:
        print(f"{ANSI['Y']}Limited to: {nb_images_to_display} rows{ANSI['W']}")
    
    print(f"{ANSI['M']}Pipeline: Overlay → Mask → Final Result{ANSI['W']}")

print(f"{ANSI['G']}3-column pipeline display function loaded{ANSI['W']}")

In [ ]:
def export_pipeline_images(images_path, masks_path, export_dir, fileid_filter=None, nb_rows=10, figsize=(15, 5), dpi=150):
    """Export images in 3-column pipeline format to files"""
    
    os.makedirs(export_dir, exist_ok=True)
    final_images = glob.glob(os.path.join(images_path, "*.png"))
    if not final_images:
        print(f"{ANSI['R']}No images found in: {images_path}{ANSI['W']}")
        return
    
    # Get FileIDs and apply filter
    file_ids = [os.path.splitext(os.path.basename(img))[0] for img in final_images]
    if fileid_filter:
        file_ids = [fid for fid in file_ids if fid not in fileid_filter]
    file_ids.sort()
    
    # Split into chunks
    chunks = [file_ids[i:i+nb_rows] for i in range(0, len(file_ids), nb_rows)]
    
    for chunk_idx, chunk in enumerate(tqdm(chunks, desc="Exporting chunks")):
        fig, axes = plt.subplots(len(chunk), 3, figsize=(figsize[0], figsize[1]*len(chunk)))
        if len(chunk) == 1:
            axes = axes.reshape(1, -1)
        
        for row_idx, file_id in enumerate(chunk):
            # Find image paths
            overlay_path = next((f for f in glob.glob(os.path.join(masks_path, "*.png")) 
                               if file_id in os.path.basename(f) and "overlay" in os.path.basename(f)), None)
            mask_path = next((f for f in glob.glob(os.path.join(masks_path, "*.png")) 
                            if file_id in os.path.basename(f) and "mask" in os.path.basename(f) and "overlay" not in os.path.basename(f)), None)
            final_path = os.path.join(images_path, f"{file_id}.png")
            
            # Display images
            for col_idx, (img_path, title) in enumerate(zip([overlay_path, mask_path, final_path], ['Overlay', 'Mask', 'Final'])):
                try:
                    if img_path and os.path.exists(img_path):
                        img = Image.open(img_path)
                        axes[row_idx, col_idx].imshow(img, cmap='gray' if img.mode == 'L' else None)
                    else:
                        axes[row_idx, col_idx].text(0.5, 0.5, f'Missing\n{title}', ha='center', va='center', 
                                                  transform=axes[row_idx, col_idx].transAxes, fontsize=12, color='red')
                except:
                    axes[row_idx, col_idx].text(0.5, 0.5, f'Error\n{title}', ha='center', va='center', 
                                              transform=axes[row_idx, col_idx].transAxes, fontsize=12, color='red')
                
                axes[row_idx, col_idx].set_title(f"{title}\n{file_id}", fontsize=10)
                axes[row_idx, col_idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(export_dir, f"pipeline_batch_{chunk_idx+1:03d}.png"), dpi=dpi, bbox_inches='tight')
        plt.close()
    
    print(f"{ANSI['G']}Exported {len(chunks)} files to {export_dir}{ANSI['W']}")

print(f"{ANSI['G']}Export function loaded{ANSI['W']}")


## 📊 **Example Usage**

### Display Function Parameters:
- **`nb_images_to_display`**: Number of rows to display (optional)
- **`filter`**: List of specific FileIDs to show (optional)

### Export Function Parameters:
- **`images_path`**: Path to final processed images
- **`masks_path`**: Path to overlay and mask images
- **`export_dir`**: Path to save exported graphs
- **`fileid_filter`**: List of FileIDs to ignore (optional)
- **`nb_rows`**: Number of rows per exported graph (default: 10)
- **`figsize`**: Figure size tuple (default: (15, 5))
- **`dpi`**: Image resolution (default: 150)

### How it works:
1. **Gets FileIDs** from final images folder
2. **Finds matching files** in masks folder for each FileID
3. **Filters out** FileIDs in `fileid_filter`
4. **Splits** FileIDs into chunks based on `nb_rows`
5. **Exports** each chunk as a separate PNG file with progress tracking

In [ ]:
# Display first 5 image rows (15 images total)
# display_pipeline_images(nb_images_to_display=5)

# Display specific FileIDs only
# display_pipeline_images(filter=['FileID_001', 'FileID_002', 'FileID_010'])

# Display all available images
display_pipeline_images()

## 🔧 **Advanced Usage Examples**

The function intelligently matches files based on FileID and provides detailed feedback:

In [ ]:
# Combined filtering and limiting
# Example: Show only first 3 rows of specific FileIDs
# display_pipeline_images(
#     nb_images_to_display=3,
#     filter=['FileID_005', 'FileID_010', 'FileID_015', 'FileID_020', 'FileID_025']
# )

# Quick preview - just 2 rows
# display_pipeline_images(nb_images_to_display=2)

# Export examples
# Export all images with 10 rows per file
# export_pipeline_images(IMAGES_PATH, MASKS_PATH, EXPORT_DIR, FILEID_FILTER, nb_rows=10)

# Export with custom settings
# export_pipeline_images(IMAGES_PATH, MASKS_PATH, EXPORT_DIR, ['FileID_001', 'FileID_002'], nb_rows=5, dpi=300)

print(f"{ANSI['C']}Uncomment the lines above to test the functions!{ANSI['W']}")
print(f"{ANSI['Y']}Remember to adjust FileIDs in the filter list to match your actual data{ANSI['W']}")